# Logit
Run through the night.... uh turns out you didn't really need to.

Gridsearch on both ready.dat and reduce.dat. Can?

In [1]:
from dataload import *

In [2]:
x_train, x_test, y_train, y_test = load_object('reduce.dat')

In [8]:

from sampler import make_pipe
samp_pipe = make_pipe(29999, levels=['dos', 'normal', 'probe'])
x_train_r, y_train_r = samp_pipe.fit_sample(x_train, y_train.attack_type)

x_train_r.shape

(149995, 35)

In [11]:
save_object([x_train_r, y_train_r],"samplexy.dat")

In [3]:
x,y = load_object("samplexy.dat")

In [5]:
x.shape

(149995, 35)

In [9]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,cohen_kappa_score,roc_auc_score
def score_test(model, x,y):
    from pandas import DataFrame
    y_pred = model.predict(x)
    labels = ['normal', 'dos', 'probe', 'r2l', 'u2r']
    return DataFrame({
        'precision' : precision_score(y, y_pred, average=None, labels=labels),
        'recall' : recall_score(y, y_pred, average=None, labels=labels),
        'F1' : f1_score(y, y_pred, average=None, labels=labels),
        'kappa' : cohen_kappa_score(y, y_pred, labels=labels) },
        # 'AUC' : roc_auc_score(y_test.attack_type, y_pred, average=None) },
        index=labels)

In [14]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
logit = LogisticRegressionCV(Cs=[1,100,500,750,900], scoring='recall', multi_class='ovr', solver='saga', random_state=4129)
%time logit.fit(x_train_r, y_train_r)

C:\Users\sharon.woo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Wall time: 9min 35s


LogisticRegressionCV(Cs=[1, 100, 500, 750, 900], class_weight=None, cv=None,
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2',
           random_state=4129, refit=True, scoring='recall', solver='saga',
           tol=0.0001, verbose=0)

In [15]:
score_test(logit,x_test,y_test.attack_type)

,F1,kappa,precision,recall
normal,0.980206,0.975847,0.995726,0.965162
dos,0.999348,0.975847,0.999948,0.998749
probe,0.917500,0.975847,0.860768,0.982238
r2l,0.080793,0.975847,0.042146,0.973451
u2r,0.025974,0.975847,0.013158,1.000000


In [16]:
logit.C_

array([100, 500, 500, 100, 100])

In [12]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

logit = LogisticRegression(C=500, multi_class='ovr', solver='saga', n_jobs=-1, random_state=4129)
%time bigmodel = logit.fit(x_train, y_train.attack_type)

C:\Users\sharon.woo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Wall time: 29min 33s


In [17]:
save_object(bigmodel,"biglogitmodel.dat")

In [16]:
len(logit.coef_)

5

In [13]:
score_test(logit,x_test,y_test.attack_type)

,F1,kappa,precision,recall
normal,0.996140,0.994802,0.994782,0.997502
dos,0.999454,0.994802,0.999750,0.999158
probe,0.954815,0.994802,0.950784,0.958881
r2l,0.114650,0.994802,0.204545,0.079646
u2r,0.500000,0.994802,0.666667,0.400000
